# Regression topics

This section will go into more detail on running regressions in Python. We already saw an example using [factor models](10_factor_models.html#factor_models), like the CAPM and Fama-French 3-factor models. 

We could spend an entire semester going over linear regression, how to put together models, how to interpret models, and all of the adjustments that we can make. In fact, this is basically what a first-semester Econometrics class is!

I will be following code examples from [Coding for Economists](https://aeturrell.github.io/coding-for-economists/econmt-regression.html), which has just about everything you need to know to do basic linear regression (OLS) in Python. I recommend giving it a read, especially if you've taken econometrics and have already seen the general ideas.

[The Effect](https://theeffectbook.net) great book for getting starting with econometrics, regression, and how to add meaning to the regressions that we're running. [Chapter 13](https://theeffectbook.net/ch-StatisticalAdjustment.html) of that book covers regression (with code in R).

You can read more about [statsmodels on their help page](https://www.statsmodels.org/dev/regression.html).

I'll be using our Zillow pricing error data in this example.

In [28]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.options.display.max_columns = None

In [29]:
housing = pd.read_csv('https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/properties_2016_sample10_1.csv')
pricing = pd.read_csv('https://raw.githubusercontent.com/aaiken1/fin-data-analysis-python/main/data/train_2016_v2.csv')

zillow_data = pd.merge(housing, pricing, how='inner', on='parcelid')
zillow_data['transactiondate'] = pd.to_datetime(zillow_data['transactiondate'], format='%Y-%m-%d')


/Users/adamaiken/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
zillow_data.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,9.071000e+03,2871.000000,0.0,5.00000,9071.000000,9071.000000,3.0,5694.000000,8948.000000,64.0,695.000000,9001.000000,8612.000000,3.000000,337.000000,695.000000,49.000000,9071.000000,993.000000,8948.000000,3076.000000,3076.000000,5574.000000,9.071000e+03,9.071000e+03,8.020000e+03,1810.0,99.000000,1685.0,9071.000000,9.071000e+03,8912.000000,9071.000000,3601.000000,9070.000000,9071.000000,5.0,1208.000000,0.0,5794.000000,280.000000,7.000000,8991.000000,2138.000000,9.022000e+03,9.071000e+03,9071.0,9.071000e+03,9071.000000,168.000000,9.009000e+03,9071.000000
mean,1.298764e+07,1.838036,NaN,516.00000,2.266233,3.013670,4.0,5.572708,2.296826,66.0,1348.981295,1767.239307,1740.108918,1408.000000,2393.350148,1368.942446,2251.428571,6049.128982,1.197382,2.228990,1.800715,342.415475,3.909760,3.400230e+07,-1.181977e+08,3.150909e+04,1.0,520.424242,1.0,261.835520,6.049436e+07,33944.006845,2511.879727,193520.398223,96547.689195,1.531364,7.0,1.004967,NaN,1.104764,290.335714,496.714286,1968.380047,1.428438,1.768673e+05,4.523049e+05,2015.0,2.763930e+05,5906.696988,13.327381,6.049368e+13,0.010703
std,1.757451e+06,3.001723,NaN,233.49197,0.989863,1.118468,0.0,1.908379,0.960557,0.0,664.508053,918.999586,880.213401,55.425626,1434.457485,709.622839,1352.034747,20.794593,0.480794,0.951007,0.598328,263.642761,3.678727,2.654493e+05,3.631575e+05,1.824345e+05,0.0,146.537109,0.0,5.781663,2.063550e+05,47178.373342,810.417898,169701.596819,412.732130,2.856603,0.0,0.070330,NaN,0.459551,172.987812,506.445033,23.469997,0.536698,1.909207e+05,5.229433e+05,0.0,3.901131e+05,6388.966672,1.796527,2.053649e+11,0.158364
min,1.071186e+07,1.000000,NaN,162.00000,0.000000,0.000000,4.0,1.000000,1.000000,66.0,49.000000,214.000000,214.000000,1344.000000,716.000000,49.000000,438.000000,6037.000000,1.000000,1.000000,0.000000,0.000000,1.000000,3.334420e+07,-1.194143e+08,4.350000e+02,1.0,207.000000,1.0,31.000000,6.037101e+07,3491.000000,1286.000000,6952.000000,95982.000000,0.000000,7.0,1.000000,NaN,1.000000,41.000000,37.000000,1885.000000,1.000000,1.516000e+03,7.837000e+03,2015.0,2.178000e+03,96.740000,7.000000,6.037101e+13,-2.365000
25%,1.157119e+07,1.000000,NaN,485.00000,2.000000,2.000000,4.0,4.000000,2.000000,66.0,938.000000,1187.000000,1173.000000,1392.000000,1668.000000,938.000000,1009.000000,6037.000000,1.000000,2.000000,2.000000,0.000000,2.000000,3.380545e+07,-1.184080e+08,5.746500e+03,1.0,435.000000,1.0,261.000000,6.037400e+07,12447.000000,1286.000000,46736.000000,96193.000000,0.000000,7.0,1.000000,NaN,1.000000,175.750000,110.500000,1953.000000,1.000000,8.028525e+04,1.926595e+05,2015.0,8.060700e+04,2828.645000,13.000000,6.037400e+13,-0.025300
50%,1.259048e+07,1.000000,NaN,515.00000,2.000000,3.000000,4.0,7.000000,2.000000,66.0,1249.000000,1539.000000,1513.000000,1440.000000,2157.000000,1257.000000,1835.000000,6037.000000,1.000000,2.000000,2.000000,430.000000,2.000000,3.401408e+07,-1.181670e+08,7.200000e+03,1.0,504.000000,1.0,261.000000,6.037621e+07,25218.000000,3101.000000,118887.000000,96401.000000,0.000000,7.0,1.000000,NaN,1.000000,248.500000,268.000000,1969.000000,1.000000,1.315530e+05,3.416920e+05,2015.0,1.910000e+05,4521.580000,1

I'll print a list of the columns, just to see what our variables are. There's a lot in this data set.

In [31]:
zillow_data.columns

Index(['parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildin

Let's run a really simple regression. Can we explain pricing errors using the size of the house? I'll take the natural log of `calculatedfinishedsquarefeet` and use that as my independent (**X**) variable. My dependent (**Y**) variable will be `logerror`. I'm taking the natural log of the square footage, in order to have what's called a "log-log" model.

In [32]:
zillow_data['ln_calculatedfinishedsquarefeet'] = np.log(zillow_data['calculatedfinishedsquarefeet'])

results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet", data=zillow_data).fit()


In [33]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     13.30
Date:                Thu, 21 Apr 2022   Prob (F-statistic):           0.000267
Time:                        12:21:17   Log-Likelihood:                 3831.8
No. Observations:                9001   AIC:                            -7660.
Df Residuals:                    8999   BIC:                            -7645.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

That's the full summary of the regression. This is a "log-log" model, so we can say that a 1% change in square footage leads to a 1.39% increase in pricing error. The coefficient is positive and statistically significant at conventional levels (e.g. 1%). 

We can pull out just a piece of this full result if we like. 

In [34]:
results.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-0.0911,0.028,-3.244,0.001,-0.146,-0.036
ln_calculatedfinishedsquarefeet,0.0139,0.004,3.647,0.000,0.006,0.021


We can, of course, include multiple **X** variables in a regression. I'll add bathroom and bedroom counts to the regression model.

In [35]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + bathroomcnt + bedroomcnt", data=zillow_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.718
Date:                Thu, 21 Apr 2022   Prob (F-statistic):           0.000159
Time:                        12:21:17   Log-Likelihood:                 3835.2
No. Observations:                9001   AIC:                            -7662.
Df Residuals:                    8997   BIC:                            -7634.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

Hey, all of my significance went away! Welcome to the world of [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity). All of these variables are very correlated, so the coefficient estimates become difficult to interpret.

Watch what happens when I just run the model with the bedroom count. The $t$-statistic is quite large again.

In [36]:
results = smf.ols("logerror ~ bedroomcnt", data=zillow_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     21.69
Date:                Thu, 21 Apr 2022   Prob (F-statistic):           3.24e-06
Time:                        12:21:18   Log-Likelihood:                 3856.7
No. Observations:                9071   AIC:                            -7709.
Df Residuals:                    9069   BIC:                            -7695.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0101      0.005     -2.125      0.0

## Indicators and categorical variables

The variables used above are measured numerically. Some are **continuous**, like square footage, while others are **counts**, like the number of bedrooms. Sometimes, though, we want to include an **indicator** for something? For example, does this house have a pool or not?

There is a variable in the data called `poolcnt`. It seems to be either missing (NaN) or set equal to 1. I believe that a value of 1 means that the house has a pool and that `NaN` means that it does not. This is bit of a tricky assumption, because `NaN` could mean no pool or that we don't know either way. But, I'll make that assumption for illustrative purposes.

In [37]:
zillow_data['poolcnt'].describe()

count    1810.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: poolcnt, dtype: float64

I am going to create a new variable, `pool_d`, that is set equal to 1 if `poolcnt >= 1` and 0 otherwise. This type of 1/0 categorical variable is sometimes called an **indicator**, or **dummy** variable. 

In [38]:
zillow_data['pool_d'] = np.where(zillow_data.poolcnt.isnull(), 0, zillow_data.poolcnt >= 1)
zillow_data['pool_d'].describe()

count    9071.000000
mean        0.199537
std         0.399674
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: pool_d, dtype: float64

I can then use this 1/0 variable in my regression.

In [39]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + pool_d", data=zillow_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.684
Date:                Thu, 21 Apr 2022   Prob (F-statistic):            0.00126
Time:                        12:21:18   Log-Likelihood:                 3831.8
No. Observations:                9001   AIC:                            -7658.
Df Residuals:                    8998   BIC:                            -7636.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

Pools don't seem to influence pricing errors. 

We can also create more general **categorical** variables. For example, instead of treating bedrooms like a count, we can create new categories for each number of bedrooms. This type of model is helpful when dealing states or regions. For example, you could turn a zip code into a categorical variable. This would allow zip codes, or a location, to explain the pricing errors. 

In Python, you can turn something into a categorical variable by using `C()` in the regression formula. 

I'll try the count of bedrooms first.

In [40]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + C(bedroomcnt)", data=zillow_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     3.118
Date:                Thu, 21 Apr 2022   Prob (F-statistic):           0.000196
Time:                        12:21:18   Log-Likelihood:                 3843.8
No. Observations:                9001   AIC:                            -7662.
Df Residuals:                    8988   BIC:                            -7569.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

And here are zip codes as a categorical variable. This is saying: Is the house in this zip code or no? If it is, the indicator for that zip code gets a 1, and a 0 otherwise. If we didn't do this, then the zip code would get treated like a numerical variable in the regression, like square footage, which makes no sense!

In [42]:
results = smf.ols("logerror ~ ln_calculatedfinishedsquarefeet + C(regionidzip)", data=zillow_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               logerror   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.300
Date:                Thu, 21 Apr 2022   Prob (F-statistic):           0.000104
Time:                        12:21:32   Log-Likelihood:                 4075.3
No. Observations:                9001   AIC:                            -7391.
Df Residuals:                    8621   BIC:                            -4691.
Df Model:                         379                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 